In [ ]:
from pathlib import Path
from omop_alchemy.cdm.base import bootstrap
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

from omop_alchemy.model.vocabulary import (
    Domain,
    Vocabulary,
    Concept_Class,
    Relationship,
    Concept,
    Concept_Ancestor,
    Concept_Relationship,
)

ATHENA_LOAD_ORDER = [
    Domain,
    Vocabulary,
    Concept_Class,
    Relationship,
    Concept,
    Concept_Ancestor,
    Concept_Relationship,
]




def db_session(engine):
    """
    SQLAlchemy session per test.
    """
    Session = sessionmaker(bind=engine, future=True)
    session = Session()
    try:
        yield session
        session.commit()
    finally:
        session.rollback()
        session.close()

def athena_vocab(engine):
    """
    Load a minimal, internally consistent Athena vocabulary
    using the real ORM CSV loader.
    """
    Session = sessionmaker(bind=engine, future=True)
    session = Session()

    base_path = (
        Path(__file__).parent
        / "fixtures"
        / "athena_source"
    )

    for model in ATHENA_LOAD_ORDER:
        csv_path = base_path / f"{model.__tablename__}.csv"
        if not csv_path.exists():
            raise RuntimeError(f"Missing vocab CSV: {csv_path}")

        model.load_csv(session, csv_path)

    session.commit()
    session.close()

    yield


In [ ]:
engine = sa.create_engine(
    "sqlite://///Users/z3061723/Documents/CODE/OMOP_Alchemy/notebooks/test.db",
    future=True,
)

bootstrap(engine, create=True)

In [ ]:
Session = sessionmaker(bind=engine, future=True)
session = Session()

In [ ]:
base_path = Path('/Users/z3061723/Documents/CODE/OMOP_Alchemy/tests/fixtures/athena_source')

In [ ]:
for model in ATHENA_LOAD_ORDER:
    model.load_csv(session, base_path / f"{model.__tablename__.upper()}.csv")

In [ ]:
from sqlalchemy.exc import IntegrityError
from omop_alchemy.cdm.base.declarative import explain_sqlite_fk_error

try:
    session.commit()
except IntegrityError as e:
    session.rollback()
    explain_sqlite_fk_error(session, e)